In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

**First we check if we are navigating to the right directory for file using listdir**

In [ ]:
os.listdir('../input/telco-customer-churn')

**we now import the required libraries**

In [ ]:
import numpy as np
import pandas as pd
import pandas_profiling as pdf
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from pylab import rcParams
%matplotlib inline
#loading the CSV
filepath = "../input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv"
data = pd.read_csv(filepath)

**we now check a portion of data to get an idea about it **

In [ ]:
data.head()

**we now check the shape i.e rows and columns**

In [ ]:
data.shape

** now we create a pie chart to 

In [ ]:
#plotting the data
sizes = data['Churn'].value_counts(sort=True)
print(sizes)
colors = ['orange', 'red']
#how much and which portion to "explode out of chart
explode=[0.0, 0.1]
rcParams['figure.figsize'] = 5,5
labels =['no', 'yes']

#plot 
plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%', shadow=True, startangle=270,)

plt.title('percentage of churn in dataset')
plt.show()

**we dont need customer ID to make prediction so we drop it**

In [ ]:
data.drop(['customerID'], axis=1, inplace=True)

**we now check the sum od null values in all columns**

In [ ]:
data.isnull().sum()

**we now check the datatypes of all columns**

In [ ]:
data.dtypes

In [ ]:
pdf.ProfileReport(data)

**we now check mean, min and max values to check any outliers**

In [ ]:
data.describe()

**we see that "TotalCharges" is in object format. and our ML model can only work with numeric data. we will use "to_numeric" fuction for that**

In [ ]:
data['TotalCharges'] = data['TotalCharges'].replace(" ", 0).astype('float32')

In [ ]:
data.isnull().sum()

**we again check the values and type of TotalCharges columns**


In [ ]:
data['TotalCharges']

**as we want only numeric values in columns so we replace 'yes' and 'No' in churn column with 1 and 0 respectively**

In [ ]:
data['Churn'].replace(to_replace='Yes', value=1, inplace=True)
data['Churn'].replace(to_replace='No', value=0, inplace=True)

**similarly we replace yes and no with 1 and 0 in other columns too**

In [ ]:
data.gender = [1 if each == 'Male' else 0 for each in data.gender]
columns_to_convert = ['Partner',
                      'Dependents',
                      'PhoneService',
                      'OnlineSecurity',
                      'OnlineBackup',
                      'DeviceProtection',
                      'TechSupport',
                      'StreamingTV',
                      'StreamingMovies',
                      'PaperlessBilling']
for item in columns_to_convert:
    data[item] = [1 if each == 'Yes' else 0 for each in data[item]]
data.head()

In [ ]:
import seaborn as sns
g1 = sns.catplot(x='Contract', y='Churn', data=data, kind='bar')
g1.set_ylabels('Churn Probability')

g2= sns.catplot(x='InternetService', y='Churn', data=data, kind='bar')
g2.set_ylabels('Churn Probability')

**People having lower tenure and higher monthly charges are tend to churn more. Also as you can see below; having month-to-month contract and fiber obtic internet have a really huge effect on churn probability.**

**in columns where there are other string values apart from yes and no we could use get_dummies() convert them to 1 and 0.**

In [ ]:
data = pd.get_dummies(data)
data.dtypes

In [ ]:
data.head()

In [ ]:
data.corr()['Churn'].sort_values()

**Customers having month-to-month contract, having fiber optic internet service and using electronic payment are tend to churn more whereas people having two-year contract and having internet service are tend to not churn.**

**now we seperate data into X and Y where X is independent variables and Y is dependent or our prediction values**

In [ ]:
Y = data['Churn'].values
X = data.drop(labels =['Churn'], axis=1)

#create train and test dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=101)


**now we import logistic regression model as it used for yes for binary prediction, in our case yes or no and fit the model the data to model using fit()**


In [ ]:
#importing logistic regerssion model
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
result = model.fit(X_train, y_train)

**now we check the accuracy of the model**

In [ ]:
from sklearn import metrics
prediction = model.predict(X_test)
#print the prdiction accuracy
print(metrics.accuracy_score(y_test, prediction))

**we see that the accuracy comes out to be pretty decent at around 80% which means we were able to predict 80% of the cases correctly but accuaracy is not the best predictor of model accuracy**

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
results = confusion_matrix(y_test, prediction)
print('Confusion Matrix: ')
print(results)
print(classification_report(y_test, prediction))

**Here we seee that the model gave recall of 0.92 which means high no of corrctly recognized class i.e correct predict of chrun cases.**

**Recall:**

**Recall can be defined as the ratio of the total number of correctly classified positive examples divide to the total number of positive examples. High Recall indicates the class is correctly recognized (small number of FN).**

**Precision:**
**To get the value of precision we divide the total number of correctly classified positive examples by the total number of predicted positive examples. High Precision indicates an example labeled as positive is indeed positive (small number of FP).**

**High recall, low precision:This means that most of the positive examples are correctly recognized (low FN) but there are a lot of false positives.**

**Low recall, high precision:This shows that we miss a lot of positive examples (high FN) but those we predict as positive are indeed positive (low FP)**

**The F-Measure will always be nearer to the smaller value of Precision or Recall.**